In [ ]:
%config InlineBackend.figure_format = 'retina'
#from comet_ml import Experiment
%load_ext autoreload
%autoreload 2
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader, Sampler
from torchvision import datasets, transforms, models
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch import optim
from torch.optim import lr_scheduler, Adam
from sklearn.metrics import confusion_matrix
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import LinearAxis, Range1d
import cv2
import matplotlib.pyplot as plt
import numpy as np
import PIL
from PIL import Image
from pathlib import Path
import time
import copy
from shutil import copyfile
import os
from torchvision.utils import save_image
from IPython.display import Markdown, display
from sklearn.model_selection import RandomizedSearchCV
from torch.optim.lr_scheduler import ReduceLROnPlateau
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
from logger import Logger
import datetime
import pandas as pd
import seaborn as sns

In [ ]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

### equal pull from classes

In [ ]:
def make_weights_for_balanced_classes(train_imgs, nclasses):   
    #only weight training dataset 
    
    class_sample_counts = [0] * nclasses                                                      
    for item in train_imgs:  
        class_sample_counts[item[1]] += 1      
    print('counts per class: ', class_sample_counts)
#     weight_per_class = [0.] * nclasses                                      
#     N = float(sum(class_sample_counts))                                                   
#     for i in range(nclasses): 
#         weight_per_class[i] = N/float(class_sample_counts[i])                                 
#     weight = [0] * len(images)                                              
#     for idx, val in enumerate(images):    
#         weight[idx] = weight_per_class[val[1]]  
        
    class_weights = 1./torch.Tensor(class_sample_counts)
    train_targets = [sample[1] for sample in train_imgs]
    train_samples_weights = [class_weights[class_id] for class_id in train_targets]

    return torch.DoubleTensor(train_samples_weights)

In [ ]:
def load_split_train_val(class_names, datadir, batch_size, show_sample=True, num_workers=32, valid_size = .8):
    
    all_transforms = transforms.Compose([transforms.Resize(224),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
    all_data_wpath = ImageFolderWithPaths(datadir,transform=all_transforms) #custom dataset that includes entire path
    
    
#     num_train = len(all_data_wpath)
#     indices = list(range(num_train))
#     split = int(np.floor(valid_size * num_train))
#     np.random.shuffle(indices)
#     train_idx, val_idx = indices[split:], indices[:split-1]
    
#     train_data = torch.utils.data.Subset(all_data_wpath, train_idx)
#     val_data = torch.utils.data.Subset(all_data_wpath, val_idx)
    
    train_length = int(valid_size* len(all_data_wpath))
    val_length = len(all_data_wpath)-train_length
    train_data, val_data = torch.utils.data.random_split(all_data_wpath,(train_length,val_length))
    #print(len(train_data), len(val_data))
    
    # For an unbalanced dataset we create a weighted sampler                       
    train_samples_weights = make_weights_for_balanced_classes(train_data.dataset.imgs, len(range(num_classes)))                                                                   
    
    train_sampler = torch.utils.data.sampler.WeightedRandomSampler(train_samples_weights, 
                                                                   len(train_samples_weights),
                                                                   replacement=True)                     
    trainloader = torch.utils.data.DataLoader(train_data.dataset, batch_size=batch_size,                              
                                            sampler = train_sampler, num_workers=num_workers, pin_memory=True)    
    
    val_sampler = SubsetRandomSampler(val_data.indices)                 
    valloader = torch.utils.data.DataLoader(val_data.dataset, batch_size=batch_size,                             
                                            sampler = val_sampler, num_workers=num_workers, pin_memory=True)  

#     val_samples_weights = make_weights_for_balanced_classes(val_data.dataset.imgs, len(range(num_classes)))                                                                   
    
#     val_sampler = torch.utils.data.sampler.WeightedRandomSampler(val_samples_weights, 
#                                                                    len(val_samples_weights),
#                                                                    replacement=True)                     
#     valloader = torch.utils.data.DataLoader(val_data.dataset, batch_size=batch_size,                              
#                                             sampler = val_sampler, num_workers=num_workers, pin_memory=True)    
    
    if show_sample:
        show_sample(train_data, train_sampler)
            
    return trainloader, valloader

In [ ]:
def show_sample(train_data, train_sampler):
    
    batch_size_sampler=20
    sample_loader = torch.utils.data.DataLoader(train_data.dataset, batch_size=batch_size_sampler, \
                                                sampler = train_sampler, num_workers=1, drop_last=True)

    data_iter = iter(sample_loader)

    images, labels, paths = data_iter.next()
    fig, ax = plt.subplots(batch_size_sampler//5, 5, figsize=(10, 8))

    for j in range(images.size()[0]):
        
        # Undo preprocessing
        image = images[j].permute(1, 2, 0).cpu().numpy()
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])

        image = std * image + mean

        # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
        image = np.clip(image, 0, 1)
        ax = ax.flatten()
        ax[j].set_title(str(class_names[labels[j]]))
        ax[j].axis('off') 
        ax[j].imshow(image)
    plt.show()

In [ ]:
def get_test_loader(datadir,
                    batch_size,
                    num_workers,
                    shuffle=True,
                    pin_memory=True):
    """
    Utility function for loading and returning a multi-process
    test iterator 
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - shuffle: whether to shuffle the dataset after every epoch.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - data_loader: test set iterator.
    """
    transforms_ = transforms.Compose([transforms.Resize(224),  #resizing helps memory usage
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    
    all_data_wpath = ImageFolderWithPaths(datadir,transform=transforms_)

    testloader = torch.utils.data.DataLoader(all_data_wpath,pin_memory=True,shuffle=shuffle,
                    batch_size=batch_size, num_workers=num_workers)  

    return testloader

In [ ]:
# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
def set_parameter_requires_grad(model, feature_extract):
    if feature_extract:
        for param in model.parameters():
            param.requires_grad = False
            

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=False):

    # Initialize these variables which will be set in this if statement. Each of these
    # variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet50
        """
        model_ft = models.resnet50(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        #mlp = nn.DataParallel(linear)
        #mlp(torch.zeros((32, 100)))
        
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg19_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [ ]:
def tensorboard_logging(logger, loss, acc, step, model): 
    
    # 1. Log scalar values (scalar summary)
    info = { 'loss': loss, 'accuracy': acc}

    for tag, value in info.items():
        logger.scalar_summary(tag, value, step+1)

    # 2. Log values and gradients of the parameters (histogram summary)
    for tag, value in model.named_parameters():
        tag = tag.replace('.', '/')
        logger.histo_summary(tag, value.data.cpu().numpy(), step+1)
        logger.histo_summary(tag+'/grad', value.grad.data.cpu().numpy(), step+1)

    # 3. Log training images (image summary)
    #         denormalize = transforms.Normalize((-1,), (1 / 0.5,))
    #         info = { 'images': demormalize(images)[:10].cpu().numpy() }

    #         for tag, images in info.items():
    #             logger.image_summary(tag, images, i+1)

In [ ]:
def train_model(model_name, savename, dataloaders_dict, epochs, num_classes, is_inception, feature_extract=False):
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    logger_train = Logger('./logs/'+current_time+'/train/')
    logger_val = Logger('./logs/'+current_time+'/val/')
    model, input_size = initialize_model(model_name=model_name, num_classes=num_classes, feature_extract=feature_extract, use_pretrained=False)
    
    def set_dropout(model, drop_rate=0.1):
        for name, child in model.named_children():
            
            if isinstance(child, torch.nn.Dropout):
                child.p = drop_rate
            set_dropout(child, drop_rate=drop_rate)
    set_dropout(model, drop_rate=0.0)
    print(model)
    
#     model.classifier = nn.Sequential(*[model.classifier()[i] for i in range(7) if i != 2 and i !=5])
#     print(model.classifier())

    #feature extract False for all layers to be updated
   
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Send the model to GPU
    if torch.cuda.device_count() > 1:
        print("Let's use", torch.cuda.device_count(), "GPUs!")
        model = nn.DataParallel(model)
    model = model.to(device)

    # Gather the parameters to be optimized/updated in this run. If we are
    #  finetuning we will be updating all parameters. However, if we are
    #  doing feature extract method, we will only update the parameters
    #  that we have just initialized, i.e. the parameters with requires_grad
    #  is True.
    params_to_update = model.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                #print("\t",name)
    #else:
        #for name,param in model.named_parameters():
            #if param.requires_grad == True:
                #print("\t",name)

    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)
    # step_size: at how many multiples of epoch you decay
    # step_size = 1, after every 1 epoch, new_lr = lr*gamma 
    # step_size = 2, after every 2 epoch, new_lr = lr*gamma 
    # gamma = decaying factor
    #scheduler = StepLR(optimizer, step_size=1, gamma=0.1)
    
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=0, verbose=True, eps=1e-05)

    # Setup the loss fxn
    criterion = nn.CrossEntropyLoss()
     
    val_acc_history = []
    train_acc_history = []
    val_loss_history = []
    train_loss_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc_val = 0.0
    since_total = time.time()
    
    step = 0
    label_counts = [0]*len(range(num_classes))
    for epoch in range(epochs):
        since_epoch = time.time()
        #print('Epoch {}/{}'.format(epoch+1,num_epochs))
        print('-' * 20)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            print('Phase: {}'.format(phase))
            totals_train = 0
            totals_val = 0
            running_loss_train = 0.0
            running_loss_val = 0.0
            running_corrects_train = 0
            running_corrects_val = 0
            
            if phase == 'train':
                model.train() 
                logger = logger_train
            else:
                model.eval()   
                logger = logger_val
            
            
            # Iterate over data.
            for i, (inputs, labels, paths) in enumerate(dataloaders_dict[phase]):
                #print(i)
                
                for n in range(len(range(num_classes))):
                    label_counts[n] += len(np.where(labels.numpy() == n)[0])
                    
#                 for n in range(len(range(num_classes))):
#                     print("batch index {}, {} counts: {}".format(
#                         i, n, (labels == n).sum()))

                
#                print('LABEL COUNT = ', label_counts)

                inputs = inputs.to(device)
                labels = labels.to(device)
                #print(inputs.device)
                
                # zero the parameter gradients
                optimizer.zero_grad() # a clean up step for PyTorch

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # makes sure to clear the intermediate values for evaluation
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward() # compute updates for each parameter
                        optimizer.step() # make the updates for each parameter                        

                
                if phase == 'train':
                    #Batch accuracy and loss statistics   
                    batch_loss_train = loss.item() * inputs.size(0)     
                    batch_corrects_train = torch.sum(preds == labels.data) 
                    #tensorboard_logging(logger, batch_loss_train, labels, batch_corrects_train, step, model)
                    
                    #for accuracy and loss statistics overall 
                    running_loss_train += loss.item() * inputs.size(0)
                    running_corrects_train += torch.sum(preds == labels.data)
                    totals_train += labels.size(0)
                    
                    if (i+1) % 5 == 0:
                        print("Training, Batch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(i+1,\
                                                                      len(dataloaders_dict[phase]), \
                                                                      batch_loss_train/labels.size(0), \
                                                                      float(batch_corrects_train)/labels.size(0)))
                    step += 1
                    
                else:
                    #Batch accuracy and loss statistics  
                    batch_loss_val = loss.item() * inputs.size(0)     
                    batch_corrects_val = torch.sum(preds == labels.data) 
                    
                    
                    #for accuracy and loss statistics overall
                    running_loss_val += loss.item() * inputs.size(0)
                    running_corrects_val += torch.sum(preds == labels.data)
                    totals_val += labels.size(0)
                    
                    if (i+1) % 3 == 0:
                        print("Validation, Batch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(i+1,\
                                                                      len(dataloaders_dict[phase]), \
                                                                      batch_loss_val/labels.size(0), \
                                                                      float(batch_corrects_val)/labels.size(0)))

            if phase == 'train':
                #epoch loss and accuracy stats    
                epoch_loss_train = running_loss_train / totals_train
                epoch_acc_train = running_corrects_train.double() / totals_train
                scheduler.step(epoch_acc_train) #reduce learning rate if not improving acc
                print("Training Epoch {}/{}, Loss: {:.3f}, Accuracy: \033[1m {:.3f} \033[0m".format(epoch+1,epochs, epoch_loss_train, epoch_acc_train))
                #tensorboard_logging(logger, epoch_loss_train, epoch_acc_train, epoch, model)
                train_acc_history.append(epoch_acc_train)
                train_loss_history.append(epoch_loss_train)
            else: 
                epoch_loss_val = running_loss_val / totals_val
                epoch_acc_val = running_corrects_val.double() / totals_val
                scheduler.step(epoch_acc_val) #reduce learning rate if not improving acc
                print("Validation Epoch {}/{}, Loss: {:.3f}, Accuracy: \033[1m {:.3f} \033[0m".format(epoch+1,epochs, epoch_loss_val, epoch_acc_val))
                #tensorboard_logging(logger, epoch_loss_val, epoch_acc_val, epoch, model)
                val_acc_history.append(epoch_acc_val)
                val_loss_history.append(epoch_loss_val)
                
                #deep copy the model
                if epoch_acc_val > best_acc_val:
                    best_acc_val = epoch_acc_val
                    best_model_wts = copy.deepcopy(model.state_dict())
                    # save/load best model weights
                    torch.save(model, savename)

        time_elapsed = time.time() - since_epoch
        print('Epoch complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    time_elapsed = time.time() - since_total
    print('All epochs comlete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return model, train_acc_history, val_acc_history, train_loss_history, val_loss_history

# MAIN

In [ ]:
def main():
    
    for batch_size in params['batch_size']:
        print('NEW BATCH SIZE: ', batch_size) 
        train_loader, val_loader = load_split_train_val(
            class_names=class_names, 
            datadir=data_dir,
            batch_size=batch_size,
            show_sample=False,
            num_workers=num_workers)

        dataloaders_dict = {'train': train_loader, 'val': val_loader}

        model_train_accs = []
        model_val_accs = []  
        model_train_loss = []
        model_val_loss = []
        for model_name in params['model_names']:  #only resnet 
            for epochs in params['max_epochs']:
                model_ft, train_acc_history, val_acc_history, train_loss_history, val_loss_history= train_model(
                    model_name,
                    params['savename'],
                    dataloaders_dict,
                    epochs, 
                    num_classes,
                    is_inception=False
                )
                
                model_val_accs.append(val_acc_history)
                model_train_accs.append(train_acc_history)
                model_train_loss.append(train_loss_history)
                model_val_loss.append(val_loss_history)
                
    return model_name, model_train_accs, model_val_accs, model_train_loss, model_val_loss, train_loader, val_loader

In [ ]:
if __name__ == '__main__':
    params = {'lr': [0.01],
          'batch_size': [512],
          'max_epochs': [40],
          'optimizer':[torch.optim.Adam, torch.optim.Adagrad, torch.optim.Adadelta, torch.optim.Adamax],
          'momentum': [0.9, 0.999], #fixed right now
          'model_names':['resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet'],
          'savename': 'save_models/512_40_all_drop0.0'}
    #model_names = ['resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet', 'inception']

    data_dir = 'cpi_data/OLYMPEX/nopad/'
    num_workers = 20  #change to # of cores available to load images
    class_names=['aggregate', 'blurry', 'column', 'fragment', 'rimed aggregate', 'rimed column', 'sphere']
    num_classes = len(class_names)

    model_name, model_train_accs, model_val_accs, model_train_loss, model_val_loss, train_loader, val_loader = main()

In [ ]:
#ACCURACY PLOT for training and validation
fig = plt.figure(figsize=(9,5))
num_epochs = 50
plt.subplot(1, 2, 1)
plt.plot(np.arange(1,(num_epochs+1)),[i.cpu().numpy()*100 for i in model_train_accs[0]], label='train')
plt.plot(np.arange(1,(num_epochs+1)),[i.cpu().numpy()*100 for i in model_val_accs[0]], label='validation')
 
plt.legend()
plt.xticks(np.arange(1, (num_epochs+1), 10.0))
plt.xlabel("Epoch")
plt.ylabel("Accuracy [%]")


#LOSS PLOT for training and validation
plt.subplot(1, 2, 2)
plt.plot(np.arange(1,(num_epochs+1)),[i for i in model_train_loss[0]], label='train')
plt.plot(np.arange(1,(num_epochs+1)),[i for i in model_val_loss[0]], label='validation')
plt.legend()
plt.xticks(np.arange(1, (num_epochs+1), 10.0))
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
#ACCURACY PLOT for training and validation
fig = plt.figure(figsize=(10,5))
num_epochs = 50
plt.subplot(1, 2, 1)
plt.plot(np.arange(1,(num_epochs+1)),[i.cpu().numpy()*100 for i in model_train_accs[0]], label='train')
plt.plot(np.arange(1,(num_epochs+1)),[i.cpu().numpy()*100 for i in model_val_accs[0]], label='validation')
plt.ylim((90.0,100.))  
plt.legend()
plt.xticks(np.arange(1, (num_epochs+1), 10.0))
plt.xlabel("Epoch")
plt.ylabel("Accuracy [%]")


#LOSS PLOT for training and validation
plt.subplot(1, 2, 2)
plt.plot(np.arange(1,(num_epochs+1)),[i for i in model_train_loss[0]], label='train')
plt.plot(np.arange(1,(num_epochs+1)),[i for i in model_val_loss[0]], label='validation')
plt.legend()
plt.xticks(np.arange(1, (num_epochs+1), 10.0))
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
#ACCURACY PLOT for training and validation
fig = plt.figure(figsize=(9,5))
num_epochs = 15
plt.subplot(1, 2, 1)
plt.plot(np.arange(1,(num_epochs+1)),[i.cpu().numpy()*100 for i in model_train_accs[0]], label='train')
plt.plot(np.arange(1,(num_epochs+1)),[i.cpu().numpy()*100 for i in model_val_accs[0]], label='validation')
plt.ylim((0.0,100.))  
plt.legend()
plt.xticks(np.arange(1, (num_epochs+1), 1.0))
plt.xlabel("Epoch")
plt.ylabel("Accuracy [%]")


#LOSS PLOT for training and validation
plt.subplot(1, 2, 2)
plt.plot(np.arange(1,(num_epochs+1)),[i for i in model_train_loss[0]], label='train')
plt.plot(np.arange(1,(num_epochs+1)),[i for i in model_val_loss[0]], label='validation')
plt.legend()
plt.xticks(np.arange(1, (num_epochs+1), 1.0))
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()

In [ ]:
params = {'axes.labelsize': 'xx-large',
         'axes.titlesize':'xx-large',
         'xtick.labelsize':'xx-large',
         'ytick.labelsize':'xx-large'}
plt.rcParams.update(params)

In [ ]:
#transfer learning method
fig = plt.figure(figsize=(12,5))
num_epochs = 15
plt.subplot(1, 2, 1)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
colors = ['r', 'b', 'g', 'c', 'k']
model_names = ['resnet50', 'alexnet', 'vgg11_bn', 'squeezenet1_0', 'densenet121']
for i, (model,train_accs,val_accs) in enumerate(zip(model_names, model_train_accs, model_val_accs)):
    plt.plot(np.arange(1,(num_epochs+1)), [i.cpu().numpy()*100 for i in train_accs[:num_epochs]], colors[i], linestyle='--')
    plt.plot(np.arange(1,(num_epochs+1)), [i.cpu().numpy()*100 for i in val_accs[:num_epochs]], colors[i], linestyle='-', label=str(model))
plt.ylim(0,100)
plt.xlim(1,num_epochs)
plt.legend(title='Model type:', loc='right', prop={'size': 10})
plt.xticks(np.arange(1, num_epochs+1, 2.0))

plt.subplot(1, 2, 2)
plt.xlabel("Epochs")
plt.ylabel("Loss")
for i, (model,train_loss, val_loss) in enumerate(zip(model_names, model_train_loss, model_val_loss)):
    plt.plot(np.arange(1,(num_epochs+1)), [i for i in train_loss[:num_epochs]], colors[i], linestyle='--')
    plt.plot(np.arange(1,(num_epochs+1)), [i for i in val_loss[:num_epochs]], colors[i], linestyle='-',label=str(model))
plt.legend(title='Model type:', loc='right', prop={'size': 10})
plt.ylim(0,2.4)
plt.xlim(1,num_epochs)
plt.xticks(np.arange(1, num_epochs+1, 2))
plt.tight_layout()
fig.savefig('cpi_data/OLYMPEX/plots/loss_acc_allmodels_reducelr_all_512_0dp.eps')
fig.savefig('cpi_data/OLYMPEX/plots/loss_acc_allmodels_reducelr_all_512_0dp.png')

In [ ]:
# Plot the training curves of validation accuracy vs. number
#  of training epochs for the transfer learning method
plt.xlabel("Epochs")
plt.ylabel("Validation Accuracy")
for model,accs in zip(model_names, model_val_accs):
    plt.scatter(np.arange(1,(num_epochs+1)/2),accs, label=str(model))
plt.ylim((0.8,1.))
plt.legend()
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.show()

In [ ]:
def visualize_model(model, class_names, dataloaders, num_images=8):
    #subplots with title as right or wrong prediction
    was_training = model.training
    model.eval()
    images_handeled = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_handeled += 1
                ax = plt.subplot(num_images//2, 2, images_handeled)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                inputs_show = inputs[j].permute(1, 2, 0)
                plt.imshow(inputs_show.cpu().numpy())

                if images_handeled == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model_ft, class_names, dataloaders=dataloaders_dict)
plt.show()

In [ ]:
class_correct = list(0. for i in range(len(class_names)))
class_total = list(0. for i in range(len(class_names)))
with torch.no_grad():
    for data in test_loader:
        images, labels, _ = data
        #images = Variable(images)
        #labels = Variable(labels)
        images = images.to(device)
        labels = labels.to(device)
        outputs = model_ft(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(class_names)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(len(class_names)):
    print('Accuracy of %5s : %2f %%' % (
        class_names[i], 100 * class_correct[i] / class_total[i]))


Remove predictions that are wrong to clean up the dataset

In [ ]:
#model_ft = model_ft.to(device)
class_names=['agg','column','junk','rimed_agg', 'rimed_column', 'spheres']
wrong_preds = []
for batch_idx, (imgs, labels, paths) in enumerate(dataloaders_dict['train']):

    #make a prediction on each batch 
    input = Variable(imgs)
    input = input.to(device)
    predictions = model_ft(input)
    preds = torch.max(predictions, 1).indices.tolist()    
    labels = labels.cpu().numpy().tolist()

    for (file, im, la, pred) in zip(paths, imgs, labels, preds):
        
        if class_names[pred] != class_names[la]: 
            inputs_show = im.permute(1, 2, 0)
            #plt.imshow(inputs_show.cpu().numpy())
            #plt.title('predicted: ' + str(class_names[pred]) + ' labeled: ' +str(class_names[la]))
            #plt.show()

            try:
                #print('removing'+file)
                os.remove(file)
                wrong_preds.append(file)
            except FileNotFoundError as not_found:
                print('in except')
                pass


In [ ]:
bad_agg = 0
bad_col = 0
bad_rimed_col = 0
bad_rimed_agg = 0
bad_junk = 0
bad_spheres = 0
for file in wrong_preds:
    if 'aggs' in file:
        bad_agg+=1
    if 'junk' in file:
        bad_junk+=1
    if 'columns' in file:
        bad_col+=1
    if 'rimed_columns' in file:
        bad_rimed_col+=1
    if 'spheres' in file:
        bad_spheres+=1
    if 'rimed_agg' in file:
        bad_rimed_agg+=1
print('bad aggs {}, bad col {}, bad rimed col {}, bad junk {}, bad spheres {}, bad rimed agg {}'\
      .format(bad_agg, bad_col, bad_rimed_col, bad_junk, bad_spheres, bad_rimed_agg))
print()

#change remove to each category total files
print('%.3f %.3f %.3f %.3f %.3f %.3f' %(bad_agg/remove, bad_col/remove, \
      bad_rimed_col/remove, bad_rimed_agg/remove, bad_junk/remove, bad_spheres/remove))

Now let's train the model on the data that has been qc'ed and have it compute gradients on all layers

In [ ]:
# Initialize the model for this run
model_val_accs_qc = []

for model in model_names:
     = Falsefeature_extract
    model_ft, input_size = initialize_model(model, num_classes, feature_extract, use_pretrained=True)
    
    #if torch.cuda.device_count() > 1:
    #    print("Let's use", torch.cuda.device_count(), "GPUs!")
    #    model_ft = nn.DataParallel(model)
    model_ft = model_ft.to(device)
    
    # Gather the parameters to be optimized/updated in this run. If we are
    #  finetuning we will be updating all parameters. However, if we are
    #  doing feature extract method, we will only update the parameters
    #  that we have just initialized, i.e. the parameters with requires_grad
    #  is True.
    params_to_update = model_ft.parameters()
        
    print("Params to learn:")
    
    if feature_extract:
        params_to_update = []
        for name,param in model_ft.named_parameters():
            if param.requires_grad:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                print("\t",name)

    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

    # Print the model we just instantiated
    #print(model_ft)
    # Setup the loss fxn
    criterion = nn.CrossEntropyLoss()

    # Train and evaluate
    
    model_ft, val_acc_qc = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model=="inception"))
    model_val_accs_qc.append(val_acc_qc)

In [ ]:
# Plot the training curves of validation accuracy vs. number
#  of training epochs for the transfer learning method
plt.xlabel("Training Epochs")
plt.ylabel("Validation Accuracy")
for model,accs in zip(model_names, model_val_accs_qc):
    plt.scatter(range(1,num_epochs+1),accs, label=str(model))
plt.ylim((0.8,1.))
plt.legend()
plt.xticks(np.arange(1, num_epochs+1, 1.0))
plt.show()

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''

    preprocess = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
    image = preprocess(image)
    return image

In [ ]:
def predict2(image_path, model, topk=6):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    img = Image.open(image_path)
    img = img.convert('RGB')
    img = process_image(img)
    
    # Convert 2D image to 1D vector
    img = np.expand_dims(img, 0)
    
    
    img = torch.from_numpy(img)
    
    model.eval()
    inputs = Variable(img).to(device)
    logits = model.forward(inputs)
    
    ps = F.softmax(logits,dim=1)
    topk = ps.cpu().topk(topk)
    
    return (e.data.numpy().squeeze().tolist() for e in topk)

In [ ]:
def view_classify(img_path, prob, classes, crystal_names):
    ''' Function for viewing an image and it's predicted classes.
    '''
    image = Image.open(img_path)
    fig, (ax1, ax2) = plt.subplots(figsize=(7, 10), ncols=1, nrows=2)
    
    ax1.set_title(crystal_names[0])
    ax1.imshow(image)
    ax1.axis('off')
    
    y_pos = np.arange(len(prob))
    ax2.barh(y_pos, prob, align='center')
    ax2.set_yticks(y_pos)
    ax2.set_yticklabels(crystal_names)
    ax2.tick_params(axis='y', rotation=45)
    ax2.invert_yaxis()  # labels read top-to-bottom
    ax2.set_title('Class Probability')
    plt.show()
    current_time = time.strftime("%Y%m%d-%H%M%S")
    fig.savefig('classify/'+current_time+'.png',bbox_inches='tight',pad_inches=.3)
    plt.close()

In [ ]:
class_names=['aggregate','blurry','columns','fragment','rimed_aggs', 'rimed_columns', 'spheres']
nb_classes = len(class_names)
data_dir = 'cpi_data/OLYMPEX/nopad/'
model = torch.load('save_models/512_50_vgg19_bn_drop0.0')
#model_ft.load_state_dict(best_model_wts)
num_classes = 7
model_name = 'vgg'
batch_size = 512
num_epochs = 10
num_workers = 20

train_loader, val_loader = load_split_train_val(
            class_names=class_names, 
            datadir=data_dir,
            batch_size=batch_size,
            show_sample=False,
            num_workers=num_workers)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Send the model to GPU
if torch.cuda.device_count() > 1:
    #print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)

for batch_idx, (imgs, labels, img_paths) in enumerate(val_loader):
    #predictions = model_ft(imgs)
    #preds = torch.max(predictions, 1).indices.tolist()    
    
    for im in img_paths:
        
        probs, classes = predict2(im, model.to(device))  
        crystal_names = [class_names[e] for e in classes]

        #if classes[0] != 3 and classes[0] != 1 and classes[0] != 2:
        view_classify(im, probs, classes, crystal_names)
    

In [ ]:
model = torch.load('save_models/512_50_vgg19_bn_drop0.0').cuda()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_loader, val_loader = load_split_train_val(
            class_names=class_names, 
            datadir=data_dir,
            batch_size=batch_size,
            show_sample=False,
            num_workers=num_workers)


In [ ]:
df_cm = pd.DataFrame(conf_matrix, index = [i for i in class_names],
                  columns = [i for i in class_names])
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True)

In [ ]:
all_preds= []
all_labels = []
for batch_idx, (imgs, labels, img_paths) in enumerate(val_loader):
    with torch.no_grad():
        
        for batch_idx, (imgs, labels, img_paths) in enumerate(val_loader):
            # get the inputs
            inputs = imgs.to(device)
            labels = labels.to(device)
        
            output = model(inputs)
            pred = torch.argmax(output, 1)
            all_preds.append(pred.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
conf_matrix = confusion_matrix(np.asarray(list(itertools.chain(*all_preds))), np.asarray(list(itertools.chain(*all_labels))))
            

In [ ]:
conf_matrix = confusion_matrix(np.asarray(list(itertools.chain(*all_labels))),np.asarray(list(itertools.chain(*all_preds))))

df_cm = pd.DataFrame(conf_matrix, index = [i for i in class_names],
                  columns = [i for i in class_names])

fig, ax = plt.subplots(figsize = (10,7))
sns.heatmap(df_cm, annot=True, fmt='g', cmap="Blues")
plt.ylabel('Predicted label')
plt.xlabel('Actual label')
fig.tight_layout()
plt.savefig('cpi_data/OLYMPEX/plots/conf_matrix.png')

In [ ]:
np.asarray(list(itertools.chain(*all_preds)))

In [ ]:
import itertools

In [ ]:
conf